### Парсинг ссылок в датафрейме urllib и извлечение необходимой метрики

In [2]:
import pandas as pd
from urllib import parse


In [4]:
df = pd.read_excel('datasets/ad_campaigns.xlsx')
df_copy = df.copy(deep=True)

In [5]:
df.head(2)

,Название группы,Фраза (с минус-словами),Продуктивность,ID объявления,Заголовок,Текст,Ссылка
0,мрт менделеевская,"""!мрт !менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...
1,мрт цао,"""мрт менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2159 entries, 0 to 2158
Data columns (total 7 columns):
Название группы            2159 non-null object
Фраза (с минус-словами)    2159 non-null object
Продуктивность             2159 non-null float64
ID объявления              2159 non-null int64
Заголовок                  2159 non-null object
Текст                      2159 non-null object
Ссылка                     2159 non-null object
dtypes: float64(1), int64(1), object(5)
memory usage: 118.2+ KB


In [7]:
df.duplicated().sum()

0

In [8]:
df.set_axis(['group', 'phrase', 'effect', 'ad_id', 'title', 'text', 'link'], axis=1, inplace=True)

In [9]:
df.rename(columns={'group':'group1', 'phrase': 'phrase1'}).head(1)

,group1,phrase1,effect,ad_id,title,text,link
0,мрт менделеевская,"""!мрт !менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...


In [10]:
url = df.loc[0, 'link']

In [11]:
parsed = parse.urlsplit(url)

In [12]:
parsed

SplitResult(scheme='https', netloc='awesome-site.ru', path='/', query='utm_source=yandex&utm_medium=cpc&utm_campaign=a825749b87&utm_content=dev_{device_type}', fragment='')

In [13]:
parsed.netloc

'awesome-site.ru'

In [14]:
qurey_one = parsed.query

In [15]:
params_dict = parse.parse_qs(qurey_one)

In [16]:
params_dict['utm_campaign'][0]

'a825749b87'

In [18]:
def campaign_name(row):
    """
    get compaing name row
    """
    parsed = parse.urlsplit(row['link'])
    params_dict = parse.parse_qs(parsed.query)
    return params_dict['utm_campaign'][0]

In [19]:
df['campaign'] = df.apply(campaign_name, axis=1)

In [20]:
df.head(1)

,group,phrase,effect,ad_id,title,text,link,campaign
0,мрт менделеевская,"""!мрт !менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87


In [21]:
df['sumbol_count_compain'] = df['campaign'].apply(lambda x:len(x))
df.head(2)

,group,phrase,effect,ad_id,title,text,link,campaign,sumbol_count_compain
0,мрт менделеевская,"""!мрт !менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87,10
1,мрт цао,"""мрт менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87,10
